In [73]:
import numpy as np
import sklearn as sk
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
from sqlite_api import *

In [74]:
def scrape(season, position):
    # column name for the dataframe to be created
    labels = ['Player', 'Team', 'Plays', 'FPT', 'Games', 'FPG', 'RushAttempts', 
              'RushYrds', 'RushTDs', 'PassAttempts', 'Complete', 'PassYrds', 'PassTDs', 'Fumbles', 'Interceptions']
    
    # build pandas dataframe for this season's data
    df = pd.DataFrame(columns=labels)
    
    # build url and request html
    url = 'http://thehuddle.com/stats/'+str(season)+'/plays_std.php?pos=' + position
    r  = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
    contents = BeautifulSoup(r.text, 'lxml')
    
    # hard-coded number of columns in table
    num_cols = 15
    table = contents.find("tbody")
    entries = table.find_all("td")
    
    # find number of rows in the table
    num_rows = int(len(entries)/15)
    
    for rowN in range(num_rows):
        cur_row = dict()
        for colN in range(len(labels)):
            cur_row[labels[colN]] = entries[rowN*num_cols + colN].text.strip()
        cur_row['Season'] = str(season)
        cur_row['Position'] = position
        df = df.append(pd.Series(cur_row), ignore_index=True)
       
    # remove fantasy rows
    del df['FPT']
    del df['FPG']
    
    return df
        #print('row ' + str(rowN) +': '+str(cur_row))

In [75]:
def scrape_kicker(season, position='PK'):
    labels = ['Player', 'Team', 'FPT', 'Games', 'FPG', 'FGoals', 
              'FGoalsMissed', 'ExtraPoints', 'ExtraPointsMissed']
    
    # build pandas dataframe for this season's data
    df = pd.DataFrame(columns=labels)
    
    # build url and request html
    url = 'http://thehuddle.com/stats/'+str(season)+'/plays_std.php?pos=' + position
    r  = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
    contents = BeautifulSoup(r.text, 'lxml')
    
    # number of columns in table
    num_cols = len(labels)
    table = contents.find("tbody")
    entries = table.find_all("td")
    
    # find number of rows in the table
    num_rows = int(len(entries)/num_cols)
    
    for rowN in range(num_rows):
        cur_row = dict()
        for colN in range(len(labels)):
            cur_row[labels[colN]] = entries[rowN*num_cols + colN].text.strip()
        cur_row['Season'] = str(season)
        cur_row['Position'] = position
        df = df.append(pd.Series(cur_row), ignore_index=True)
     
    del df['FPT']
    del df['FPG']
    
    return df

In [76]:
def scrape_defense(season, position='DF'):
    labels = ['City', 'Team', 'FPT', 'Games', 'FPG', 'Sacks', 'FRecoveries', 
              'Interceptions', 'TDs', 'Safeties', 'RushYrdsAllowed', 'PassYrdsAllowed', 'TotalYrdsAllowed']
    
    # build pandas dataframe for this season's data
    df = pd.DataFrame(columns=labels)
    
    # build url and request html
    url = 'http://thehuddle.com/stats/'+str(season)+'/plays_std.php?pos=' + position
    r  = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
    contents = BeautifulSoup(r.text, 'lxml')
    
    # number of columns in table
    num_cols = len(labels)
    table = contents.find("tbody")
    entries = table.find_all("td")
    
    # find number of rows in the table
    num_rows = int(len(entries)/num_cols)
    
    for rowN in range(num_rows):
        cur_row = dict()
        for colN in range(len(labels)):
            cur_row[labels[colN]] = entries[rowN*num_cols + colN].text.strip()
        cur_row['Season'] = str(season)
        cur_row['Position'] = position
        df = df.append(pd.Series(cur_row), ignore_index=True)
    
    del df['FPT']
    del df['FPG']
    
    return df

In [77]:
conn = sqlite3.connect('fantasy.db')

c = conn.cursor()

In [78]:
c.execute('DROP TABLE PlayerSeason')
c.execute('DROP TABLE KickerSeason')
c.execute('DROP TABLE DefenseSeason')

In [79]:
c.execute('''CREATE TABLE PlayerSeason
             (Player VARCHAR(40), 
             Team VARCHAR(3), 
             Plays SMALLINT, 
             Games SMALLINT,
             RushAttempts SMALLINT,
             RushYrds SMALLINT,
             RushTDs SMALLINT,
             PassAttempts SMALLINT, 
             Complete SMALLINT, 
             PassYrds SMALLINT, 
             PassTDs SMALLINT, 
             Fumbles SMALLINT, 
             Interceptions SMALLINT,
             Position CHARACTER(2),
             Season CHARACTER(4),
             UNIQUE (Player, Season, Position, Team))''')

In [80]:
c.execute('''CREATE TABLE KickerSeason
            (Player VARCHAR(40),
            Team VARCHAR(3),
            Games SMALLINT,
            FGoals SMALLINT,
            FGoalsMissed SMALLINT,
            ExtraPoints SMALLINT,
            ExtraPointsMissed SMALLINT,
            Position CHARACTER(2),
            Season CHARACTER(4),
            UNIQUE (Player, Season))''')

In [81]:
c.execute('''CREATE TABLE DefenseSeason
            (City VARCHAR(20),
            Team VARCHAR(3),
            Games SMALLINT,
            Sacks SMALLINT,
            FRecoveries SMALLINT,
            Interceptions SMALLINT,
            TDs SMALLINT,
            Safeties SMALLINT,
            RushYrdsAllowed SMALLINT,
            PassYrdsAllowed SMALLINT,
            TotalYrdsAllowed SMALLINT,
            Position CHARACTER(2),
            Season CHARACTER(4),
            UNIQUE (Team, Season))''')

In [82]:
def update_players(conn, df):
    tuple_list = []
    
    c = conn.cursor()
    for ndx in range(len(df)):
        try:
            c.execute('INSERT INTO PlayerSeason VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)', tuple(df.iloc[ndx])) 
        except:
            print(str(df.iloc[ndx]['Player'])+' '+str(df.iloc[ndx]['Season'])+' already in table')
        
    conn.commit()

In [83]:
def update_kickers(conn, df):
    tuple_list = []
    
    c = conn.cursor()
    for ndx in range(len(df)):
        try:
            c.execute('INSERT INTO KickerSeason VALUES (?,?,?,?,?,?,?,?,?)', tuple(df.iloc[ndx])) 
        except:
            print(str(df.iloc[ndx]['Player'])+' '+str(df.iloc[ndx]['Season'])+' already in table')
        
    conn.commit()

In [84]:
def update_defense(conn, df):
    tuple_list = []
    
    c = conn.cursor()
    for ndx in range(len(df)):
        try:
            c.execute('INSERT INTO DefenseSeason VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)', tuple(df.iloc[ndx])) 
        except:
            print(str(df.iloc[ndx]['City'])+' '+str(df.iloc[ndx]['Season'])+' already in table')
        
    conn.commit()

In [85]:
pos = ['QB', 'WR', 'RB', 'TE']

conn = sqlite3.connect('fantasy.db')

# loop through all seasons available
for year in range(2006, 2017):
    # loop through all positions
    for cur_pos in pos:
        update_players(conn, scrape(year, cur_pos))

conn.close()

In [86]:
conn = sqlite3.connect('fantasy.db')

for year in range(2006, 2017):
    update_kickers(conn, scrape_kicker(year))
    
conn.close()

In [87]:
conn = sqlite3.connect('fantasy.db')

for year in range(2006, 2017):
    update_defense(conn, scrape_defense(year))
    
conn.close()

In [90]:
select_to_df('fantasy.db', 'PlayerSeason', ['Player', 'Team', 'Season', 'PassYrds', 'RushYrds']).head()

,Player,Team,Season,PassYrds,RushYrds
0,Peyton Manning,IND,2006,4397.0,36.0
1,Drew Brees,NO,2006,4418.0,32.0
2,Jon Kitna,DET,2006,4208.0,156.0
3,Michael Vick,ATL,2006,2474.0,1039.0
4,Carson Palmer,CIN,2006,4035.0,37.0


In [89]:
for row in c.execute('SELECT * FROM PlayerSeason WHERE Season = 2011'):
    print(row)

('Drew Brees', 'NO', 678, 16, 21, 86, 1, 657, 468, 5476, 46, 1, 14, 'QB', '2011')
('Aaron Rodgers', 'GB', 562, 15, 60, 257, 3, 502, 343, 4643, 45, 0, 6, 'QB', '2011')
('Tom Brady', 'NE', 654, 16, 43, 109, 3, 611, 401, 5235, 39, 2, 12, 'QB', '2011')
('Cam Newton', 'CAR', 644, 16, 126, 706, 14, 517, 310, 4051, 21, 2, 17, 'QB', '2011')
('Matthew Stafford', 'DET', 685, 16, 22, 78, 0, 663, 421, 5038, 41, 1, 16, 'QB', '2011')
('Eli Manning', 'NYG', 624, 16, 35, 15, 1, 589, 359, 4933, 29, 4, 16, 'QB', '2011')
('Philip Rivers', 'LAC', 608, 16, 26, 36, 1, 582, 366, 4624, 27, 5, 20, 'QB', '2011')
('Matt Ryan', 'ATL', 603, 16, 37, 84, 2, 566, 347, 4177, 29, 3, 12, 'QB', '2011')
('Tony Romo', 'DAL', 544, 16, 22, 46, 1, 522, 346, 4184, 31, 3, 10, 'QB', '2011')
('Mark Sanchez', 'NYJ', 580, 16, 37, 103, 6, 543, 308, 3474, 26, 8, 18, 'QB', '2011')
('Ryan Fitzpatrick', 'BUF', 625, 16, 56, 215, 0, 569, 353, 3832, 24, 2, 23, 'QB', '2011')
('Michael Vick', 'PHI', 498, 13, 75, 590, 1, 423, 253, 3303, 18, 4

In [22]:
type(tuple(df.iloc[4])[4])

str

In [22]:
conn.close()